In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2
import os

In [ ]:
# severity codes

severity_code = {'MILD': 0,
                'MODERATE': 1,
                'NORMAL': 2,
                'SEVERE': 3}

In [ ]:
# specifying file locations

source_folder_train = 
source_folder_val =     
channels = 3

In [ ]:
# adding the image arrays to an array

resized_images_train = [
    cv2.resize(cv2.imread(os.path.join(source_folder_train, severity, video, image)), (250, 250))
    for severity in os.listdir(source_folder_train)
    for video in os.listdir(os.path.join(source_folder_train, severity))
    for image in os.listdir(os.path.join(source_folder_train, severity, video))
]

# formatting the array

num_samples = len(resized_images_train)
input_shape = (num_samples, 250, 250, channels)
input_array_train = np.empty(input_shape, dtype=np.uint8)

In [ ]:
# adding the corresponding severity codes to the array

input_array_train = np.array(resized_images_train)
print(input_array_train.shape)
x_train = input_array_train

y_train = [
    severity_code[severity]
    for severity in os.listdir(source_folder_train)
    for video in os.listdir(os.path.join(source_folder_train, severity))
    for image in os.listdir(os.path.join(source_folder_train, severity, video))
]

In [ ]:
# adding the image arrays to an array

resized_images_val = [
    cv2.resize(cv2.imread(os.path.join(source_folder_val, severity, video, image)), (250, 250))
    for severity in os.listdir(source_folder_val)
    for video in os.listdir(os.path.join(source_folder_val, severity))
    for image in os.listdir(os.path.join(source_folder_val, severity, video))
]

# formatting the array

num_samples = len(resized_images_val)
input_shape = (num_samples, 250, 250, channels)
input_array = np.empty(input_shape, dtype=np.uint8)

In [ ]:
# adding the corresponding severity codes to the array

input_array_val = np.array(resized_images_val)
print(input_array.shape)
x_val = input_array

y_val = [
    severity_code[severity]
    for severity in os.listdir(source_folder_val)
    for video in os.listdir(os.path.join(source_folder_val, severity))
    for image in os.listdir(os.path.join(source_folder_val, severity, video))
]

In [ ]:
# randomly oversampling all the training images to have the same number of images

ros = RandomOverSampler(random_state=42)
x_train_2d = x_train.reshape(x_train.shape[0], -1)
X_resampled, y_resampled = ros.fit_resample(x_train_2d, y_train)
x_resampled_original = X_resampled.reshape(-1, x_train.shape[1], x_train.shape[2], x_train.shape[3])

In [ ]:
# applying one-hot encoding on the class values

encoder = OneHotEncoder(sparse=False)
y_val = encoder.fit_transform(np.array(y_val).reshape(-1,1))
y_train = encoder.fit_transform(np.array(y_resampled).reshape(-1,1))

In [ ]:
# normalizing the image arrays

x_resampled_original = x_resampled_original/255.0
x_val = x_val/255.0

In [ ]:
# defining the 2-layer CNN model (no regularization)

model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(250, 250, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(4, activation='softmax'))

In [ ]:
opt = tf.keras.optimizers.AdamW(learning_rate=0.0005)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

print('Model Details are : ')
print(model.summary())

In [ ]:
epochs = 10
batch_size = 8

history = model.fit(x_resampled_original, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val))

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy 2-layer (no regularization)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model accuracy 2-layer (no regularization)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# defining the 2-layer CNN model (L1 regularization, no L2 regularization)

model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(250, 250, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu', kernel_regularizer=tf.keras.regularizers.l1(0.001)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(4, activation='softmax'))

In [ ]:
opt = tf.keras.optimizers.AdamW(learning_rate=0.0005)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

print('Model Details are : ')
print(model.summary())

In [ ]:
epochs = 10
batch_size = 8

history = model.fit(x_resampled_original, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val))

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy 2-layer (L1 regularization)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model accuracy 2-layer (L1 regularization)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# defining the 2-layer CNN model (no L1 regularization, L2 regularization)

model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(250, 250, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(4, activation='softmax'))

In [ ]:
opt = tf.keras.optimizers.AdamW(learning_rate=0.0005)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

print('Model Details are : ')
print(model.summary())

In [ ]:
epochs = 10
batch_size = 8

history = model.fit(x_resampled_original, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val))

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy 2-layer (L2 regularization)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model accuracy 2-layer (L2 regularization)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# defining the 2-layer CNN model (L1 regularization, L2 regularization)

model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(250, 250, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu', kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.001, l2=0.001)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(4, activation='softmax'))

In [ ]:
opt = tf.keras.optimizers.AdamW(learning_rate=0.0005)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

print('Model Details are : ')
print(model.summary())

In [ ]:
epochs = 10
batch_size = 8

history = model.fit(x_resampled_original, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val))

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy 2-layer (L1 + L2 regularization)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model accuracy 2-layer (L1 + L2 regularization)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# defining the 1-layer CNN model (no regularization)

model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(250, 250, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(4, activation='softmax'))

In [ ]:
opt = tf.keras.optimizers.AdamW(learning_rate=0.0005)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

print('Model Details are : ')
print(model.summary())

In [ ]:
epochs = 10
batch_size = 8

history = model.fit(x_resampled_original, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val))

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy 1-layer (no regularization)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model accuracy 1-layer (no regularization)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()